In [20]:
import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

In [21]:
# Taking API keys from .env file
load_dotenv()

True

In [3]:
'''GOOGLE_API_KEY= os.getenv("GOOGLE_API_KEY")

os.environ['GOOGLE_API_KEY']= GOOGLE_API_KEY'''

'GOOGLE_API_KEY= os.getenv("GOOGLE_API_KEY")\n\nos.environ[\'GOOGLE_API_KEY\']= GOOGLE_API_KEY'

In [7]:
#Loading the model and test it

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage


llm= ChatGoogleGenerativeAI(model='gemini-pro', convert_system_message_to_human=True)

message= [SystemMessage(content='Hi you are nice bot'),
            HumanMessage(content= 'Hi!, how are you buddy?')]

llm.invoke(message)


AIMessage(content='I am not a bot, I am a human language model trained by Google. I am doing well, thank you for asking. How are you?', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-b373fa8e-393a-44e2-9b94-90f2a74348c4-0')

In [10]:
## Use output parser

from langchain_core.output_parsers import StrOutputParser

# since StrOutputparser is class need to create a method parser
parser= StrOutputParser()

response= llm.invoke(message)

parser.invoke(response)


'I am a large language model, trained by Google. I do not have feelings or emotions, and therefore I do not have the capacity to be "nice" or to have "buddies". I do not experience subjective states such as happiness, sadness, or friendship.\n\nI am designed to understand and respond to a wide range of natural language inputs, and to generate human-like text. I am still under development, but I am constantly learning and improving.\n\nMay I help you with something else today?'

### LCEL

In [11]:
# Chaining with LCEL

chain= llm | parser

chain.invoke(message)

'Thank you for the compliment! I am doing well, thank you for asking. I am a friendly bot designed to help people with their writing. I can help you with grammar, spelling, and punctuation, as well as provide suggestions for improving your writing style. I can also help you find information on the web and translate text between different languages. Is there anything I can help you with today?'

### Now lets start with prompting

In [12]:
from langchain_core.prompts import ChatPromptTemplate

# Create the prompt template
prompt_template = ChatPromptTemplate.from_messages(
    [("system","translate the following into {language}"),
     ("user","{text}")]
)

In [13]:
prompt = prompt_template.invoke({"language":"french","text":"i am learning langchain with sunny sir."})
result=llm.invoke(prompt)

parser.invoke(result)

"J'apprends LangChain avec Monsieur Sunny."

### LCEL with prompt

In [14]:
### Using LCEL


chain=  prompt_template | llm | parser

chain.invoke({"language": "tamil", "text": "It is my pleasure"})

'எனது மகிழ்ச்சி'

### TavilySearch 
3rd party API act as tools in GenAI

In [25]:
TAVILY_API_KEY= os.getenv("TAVILY_API_KEY")

os.environ['TAVILY_API_KEY']= TAVILY_API_KEY

In [27]:
from tavily import TavilyClient, MissingAPIKeyError

try:
    tavily_client = TavilyClient(api_key=TAVILY_API_KEY)
    response = tavily_client.search("Who is Leo Messi?")
    print(response)
except MissingAPIKeyError:
    print("API key is missing. Please provide a valid API key.")

{'query': 'Who is Leo Messi?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Leo Messi - Official FC Barcelona Website', 'url': 'https://www.fcbarcelona.com/en/card/2214377/leo-messi', 'content': "Notifications Center\nNotifications Center\nFirst Team\nClub\nTickets & Museum\nCulers\nDownload the official FC Barcelona App\nLeo Messi\nLeo Messi's footballing career started in 1995 at Newell's Old Boys, where he played until the year 2000. Messi’s individual achievements are also unprecedented: six Ballon d’Or awards, six times Champions League top scorer, six times Golden Shoe winner, eight times ‘Pichichi’ (top scorer) in La Liga, Ballon d’Or winner at the 2014 World Cup, FIFA World Player of the Year in 2009 and FIFA The Best winner in 2019.\n He has also played in six Copa Américas (2007, 2011, 2015, 2016, 2019, 2021), losing in the final in both 2015 and 2016 to Chile on penalties before finally claiming a major honour for his country with the win

### Agents

In [33]:
# langgraph packages

from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

# import tavilysearch tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage

In [34]:
memory= MemorySaver()

# TavilySearch is a 3rd party API for answering realtime scenario's
search=TavilySearchResults(max_results=2)
# In GenAI 3rd party API's are tools, so we are passing inside list and declare as tools
tools=[search]

In [35]:
# Agents

agent_executor=create_react_agent(llm,tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

NotImplementedError: 

In [18]:
for chunk in agent_executor.stream({"messages":[HumanMessage(content="hi im sunny! and i live in bangalore")]}, config):
    print(chunk)
    print("----")

NameError: name 'agent_executor' is not defined

In [19]:
for chunk in agent_executor.stream({"messages": [HumanMessage(content="whats the weather where I live?")]}, config):
    print(chunk)
    print("----")

NameError: name 'agent_executor' is not defined